## 절차

1. 연구목표 설정
2. 데이터 획득(수집)
 - https://play.google.com/store/apps/details?id=com.nhn.android.webtoon&showAllReviews=true
 - https://play.google.com/store/apps/details?id=net.daum.android.webtoon&showAllReviews=true
 - selenium 이용
 - 페이지로드되면, END를 작동시켜 밑 밑으로 내린다 -> 10번(임의설정), 3초간 대기
 - 리뷰를 수집해온다
  > 이름       
  > 별점       
  > 등록일     
  > 내용       
  > 도움이됬음 
  
3. 데이터 준비(정제, 변환, 통합)
  > 정규식으로 필터링(욕설, 기타 표현에 대한 정책)
  > 파생변수 -> 형태소, 점수 지표를 이용한 계산, 사전에 없는 표현에 대한 유사어검사(확장성) 등
4. 데이터 탐색(시각화, 통찰과정)
  > 워드클라우드, 바차트, 히스토그램, 산포도표현
5. 생략
6. 시스템통합(사전의 재구축(선순환구조) 및 반영, 서비스)


In [80]:
from selenium import webdriver as wd
import urllib

In [104]:
target_urls = [
    'https://play.google.com/store/apps/details?id=com.nhn.android.webtoon&showAllReviews=true',
    'https://play.google.com/store/apps/details?id=net.daum.android.webtoon&showAllReviews=true'
]

In [106]:
driver = wd.Chrome('../tool/chromedriver75.0.3770.140.exe')

In [112]:
from selenium.webdriver.common.keys import Keys
def getData(app_name):
    # 타겟 사이트별 데이터 수집 작업
    # 스크롤바를 밑으로 내린다 (10번)
    for n in range(10):
        # selenium.webdriver.common.keys.Keys
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        # ajax 통해서 리뷰가 로드될것이다
        time.sleep(3)
        #break
        
    reviews = driver.find_elements_by_css_selector('.W4P4ne>div:nth-child(2)>div>div') # W4P4ne의 자식들
    # 전체 리뷰수
    print( len(reviews) )
    tmp = list()
    try:
        for div in reviews:
            # 리뷰를 하나씩 꺼내서
            name    = div.find_element_by_class_name('X43Kjb')
            star    = div.find_element_by_css_selector('.pf5lIe>div')
            regi    = div.find_element_by_class_name('p2TkOb')
            content = div.find_element_by_css_selector('.UD7Dzf>span')
            good    = div.find_element_by_class_name('jUL89d.y92BAb')
            dic     = {
                'name':name.text,
                'star':star.get_attribute('aria-label'),
                'regi':regi.text,
                'content':content.text,
                'good':good.text,
                'app_name':app_name
            }
            tmp.append( dic )
    except Exception as e:
        print('실제로는 데이터가 없으나 있는것으로 처리되서 커트')
    return tmp
    
    
    
    
    # 수집하고자하는 내용들을 완벽하게 순서대로 세트를 유지하고, class명이 모두 동일
    # 여기서는 개별적으로 구해서 순서대로 처리
    # 후기가 있어서 수치가 틀어진다 -> 보정이 필요
    # 이름   
    #names    = driver.find_elements_by_class_name('X43Kjb')
    # 별점 
    #stars    = driver.find_elements_by_css_selector('.pf5lIe>div')
    # 등록일
    #regis    = driver.find_elements_by_class_name('p2TkOb')
    # 내용  
    #contents = driver.find_elements_by_css_selector('.UD7Dzf>span')
    # 도움이됬음 
    #goods    = driver.find_elements_by_class_name('jUL89d.y92BAb')
    #print( len(names),len(stars),len(regis),len(contents),len(goods)  )  

In [113]:
import time
outputs   = list()
app_names = ['네이버웹툰','다음웹툰']
for idx, target_url in enumerate(target_urls):    
    driver.get( target_url )
    outputs.extend( getData( app_names[idx] ) )
    time.sleep(4)
    #break

201
실제로는 데이터가 없으나 있는것으로 처리되서 커트
201
실제로는 데이터가 없으나 있는것으로 처리되서 커트


In [114]:
len(outputs)

378

In [115]:
outputs[-1:]

[{'name': '갑쬬히',
  'star': '별표 5개 만점에 1개를 받았습니다.',
  'regi': '2018년 12월 28일',
  'content': '있자나요... 1000캐시 이벤트하길레 보고 캐시를 받는데 받는데도 게속 0캐시에요 ㅠㅠ',
  'good': '1',
  'app_name': '다음웹툰'}]

In [116]:
import pandas as pd
import numpy as np

In [147]:
df = pd.DataFrame.from_dict( outputs )
df.head(2)

,app_name,content,good,name,regi,star
0,네이버웹툰,네이버 웹툰은 컷마다 볼 수 있는 댓글과 보기편한 점이 있어서 좋지만 다른 새로운 ...,5,2D투디,2019년 8월 16일,별표 5개 만점에 3개를 받았습니다.
1,네이버웹툰,아니;;장난하시냐고요--여러분 이거요기요 첫주문웬만하면하지마세요.쿠키130개?절대안...,86,tv부기,2019년 8월 11일,별표 5개 만점에 1개를 받았습니다.


In [148]:
# star에서 점수만 추출
sample = '별표 5개 만점에 3개를 받았습니다.'
import re

In [149]:
p = re.compile( '\d+' ) # \d+: [0-9]+ 와 같은 표현이다

In [150]:
p.findall( sample )[1]

'3'

In [151]:
# star값들을 실제 점수로만 변경한다
df['star'] = df['star'].apply( lambda x : p.findall( x )[1] )

In [154]:
df.dtypes

app_name    object
content     object
good        object
name        object
regi        object
star        object
dtype: object

In [177]:
# regi에서 날짜를 YYYY-mm-dd 형태를 날짜 포멧팅으로 처리
# step1. 년도, 월, 일 값들을 일단 뽑아낸다
# '2019년 8월 16일' => (\d+).+(\d+).+(\d+).   .(점)은 줄바꿈을 뺀 나머지를 뜻한다
p        = re.compile( '(\d+)\w\s(\d+)\w\s(\d+)\w' )
sample   = "2019년 8월 16일"
#data_raw = p.search(sample).groups()
# 2019-08-16  replace를 사용해도 된다(관련 코드가 뭔지 확인할 것)
p.sub( '\g<1>-\g<2>-\g<3>', sample )

'2019-8-16'

In [178]:
import datetime

In [179]:
datetime.datetime.strptime( '2019-08-16', '%Y-%m-%d' ).date()

datetime.date(2019, 8, 16)

In [ ]:
# df에 regi 항목의 데이터를 datetime.date(2019, 8, 16) 타입으로 치환하시오

In [181]:
df['regi'] = df['regi'].apply( lambda x : datetime.datetime.strptime( p.sub( '\g<1>-\g<2>-\g<3>', x ), '%Y-%m-%d' ).date() )

In [182]:
df.head(2)

,app_name,content,good,name,regi,star
0,네이버웹툰,네이버 웹툰은 컷마다 볼 수 있는 댓글과 보기편한 점이 있어서 좋지만 다른 새로운 ...,5,2D투디,2019-08-16,3
1,네이버웹툰,아니;;장난하시냐고요--여러분 이거요기요 첫주문웬만하면하지마세요.쿠키130개?절대안...,86,tv부기,2019-08-11,1


In [ ]:
# contents에서 형태소 분석을 위한 영어 제거 혹은 대체?
